In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# parameter
learning_rate = 0.01
batch_size = 1000
drop_out = 0.7
training_epoch = 20

In [5]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

X_input = tf.reshape(X, shape=[-1, 28, 28, 1])

W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]), tf.float32)
L1 = tf.nn.conv2d(X_input, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

W2 = tf.Variable(tf.random_normal([5, 5, 32, 64]), tf.float32)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

L2 = tf.layers.flatten(L2)

W3 = tf.Variable(tf.random_normal([7*7*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))
L3 = tf.matmul(L2, W3) + b3
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([1024, 10]))
b4 = tf.Variable(tf.random_normal([10]))

L4 = tf.matmul(L3, W4) + b4

hypothesis = tf.nn.softmax(L4)

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=L4, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
prediction = tf.argmax(hypothesis, axis=1)
actual = tf.argmax(Y, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, actual), tf.float32))

In [7]:
tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)
summaries = tf.summary.merge_all()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('logs/05_Convolution_Neural_Network_With_Mnist_Dataset', sess.graph)
    num_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(training_epoch):
        avg_cost = 0
        for b in range(num_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _, accuracy_val, summary_ = sess.run([cost, optimizer, accuracy, summaries], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: drop_out})
            avg_cost += cost_val / num_batch
        print("epoch {0}, cost = {1:.5f}, accuracy = {2:.5f}".format(epoch, cost_val, accuracy_val))
        summary_writer.add_summary(summary_, global_step=epoch)
    accuracy_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}) 
    print("result, accuracy = {:.5f}".format(accuracy_val))

epoch 0, cost = 1423.18079, accuracy = 0.90300
